In [20]:
import numpy as np
import IPython
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as color
import soundfile
import scipy.signal as sig
import sklearn
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import os
from os import listdir
from os.path import isfile, join
import Augmentor

In [21]:
file_path = 'nn06f_20180806_000000.wav'

# you can use scipy- import scipy.io.wavfile as siow - [samp, data] = siow.read(file_path, mmap=False)  
samples,sampling_rate=soundfile.read(file_path)

#replace this with X = pd.read_csv(r'..\annotations\training.txt', sep="\t", header=0)

X = pd.read_csv('nn_ele_hb_00-24hr_TrainingSet_v2.txt', sep="\t", header=0) 


offset=5 #5 sec offset

num_rows= np.shape(X[X['filename']==file_path])[0]
ref_table=X[X['filename']==file_path]

fmax = ref_table['High Freq (Hz)'].values.tolist()
t_start=ref_table[ref_table.columns[2]][0:num_rows+1]-offset
t_end=ref_table[ref_table.columns[2]][0:num_rows+1]+ref_table[ref_table.columns[4]][0:num_rows+1]+offset



data = []

for i,j in zip(t_start.values.tolist(),t_end.values.tolist()):
    data.append(samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate])

In [22]:
import sklearn.preprocessing as sk

In [37]:
def normalize(x, axis=0):
    return sk.minmax_scale(x, axis=axis)


def spec_plot2_one(s,samp,filename,i):

    [b,a] = sig.butter(4,[10/samp, 100/samp],'bandpass','ba') 
    data1 = sig.filtfilt(b, a, s)
    data_norm=normalize(data1)
    
    fig, axs = plt.subplots(figsize=(14, 5)) #facecolor='w', edgecolor='k')
        
    [f, t, Sxx] = sig.spectrogram(data_norm, samp,'hann',nperseg=samp,noverlap=samp*0.9,nfft = samp*3)
    cmap = plt.get_cmap('Spectral') #magma
    norm = color.Normalize(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
    
    axs.pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)  
    axs.set_ylim(10,70)
    
    plt.tight_layout()
    plt.ioff()
    fig.savefig('trainingimages/'+ filename + str(i)+'.png', bbox_inches='tight')
    plt.close()    

In [38]:
try:
    os.mkdir('trainingimages')
except FileExistsError:
    pass

In [39]:
filename=ref_table['filename'].unique()[0].strip('.wav')

for i in range(len(data)):
    spec_plot2_one(data[i],sampling_rate,filename+str(fmax[i]),i)

<ipython-input-37-c695b3bcfec4>:17: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  axs.pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)
<ipython-input-37-c695b3bcfec4>:17: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  axs.pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)
<ipython-input-37-c695b3bcfec4>:17: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals wit

In [40]:
p = Augmentor.Pipeline(source_directory= "trainingimages",output_directory='transformedimages')

Initialised with 13 image(s) found.
Output directory set to trainingimages\transformedimages.

In [41]:
p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=8)
p.rotate180(probability=0.5)
p.zoom_random(0.4, percentage_area=0.7)
p.skew_tilt(0.6,0.3)
p.random_contrast(probability=0.45, min_factor=0.6, max_factor=1)
p.random_brightness(probability=0.5, min_factor=0.6, max_factor=1)
p.process()

Processing <PIL.Image.Image image mode=RGBA size=1000x352 at 0x2499AF81CA0>: 100%|███████████████████| 13/13 [00:00<00:00, 28.52 Samples/s]
